In [1]:
import pandas as pd 
import numpy as np

import seaborn as sn
import confusion_matrix_pretty_print
from confusion_matrix_pretty_print import plot_confusion_matrix_from_data
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix,classification_report,precision_score
import keras

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Softmax, Dropout

In [2]:
tripleData = pd.read_csv('../../triple42702_ShuffledPaired.csv')

In [3]:
fold = 10 
interval = int(42702/fold)
auprListpositive1 = []
aucListpositive1 = []

auprListmines1 = []
aucListmines1 = []

In [13]:

for split1 in range(10):
    split2 = split1 + 1
    
    trainIndex = list(range(0,split1*interval))
    trainIndex.extend(list(range(split2*interval,42702)))

    if split1==9:
        testIndex = list(range(split1*interval,42702))
    else:
        testIndex = list(range(split1*interval,split2*interval))

    dataTrain = tripleData.iloc[trainIndex,:]
    dataTest = tripleData.iloc[testIndex,:]
    print(min(testIndex),max(testIndex))


    X_train, X_test = dataTrain.values[:,3:], dataTest.values[:,3:]
    y_train, y_test = dataTrain.values[:,2].astype(int), dataTest.values[:,2].astype(int)
    trainNum = len(X_train)
    testNum = len(X_test)

    #reshape data to fit model
    X_train = X_train.reshape(trainNum,16,71,1).astype('float32')
    X_test = X_test.reshape(testNum,16,71,1).astype('float32')

    y_train = y_train + 1
    y_test  = y_test + 1
    y_train = y_train / 2
    y_test  = y_test / 2
    # print(y_train[0], y_test[0])

    from keras.utils import to_categorical
    #one-hot encode target column
    y_train = to_categorical(y_train).astype(int)
    y_test = to_categorical(y_test).astype(int)
    # y_test[0]
    #create model
    model = Sequential()
    #add model layers
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    model.add(Conv2D(128, kernel_size=4, activation='relu', input_shape=(16,71,1)))
    # model.add(Conv2D(64, kernel_size=2, activation='relu'))

    model.add(Conv2D(32, kernel_size=4, activation='relu'))
    # model.add(Conv2D(16, kernel_size=2, activation='relu'))
    model.add(Conv2D(8, kernel_size=4, activation='relu'))
    model.add(Flatten())
    model.add(Dense( 1024, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense( 64, activation='relu'))
    model.add(Dense( 2, activation='relu'))
    # model.add(Softmax())
    model.summary()


    #compile model using accuracy to measure model performance
    from keras import optimizers
    from keras import metrics as kmetr


    adam = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    # model.compile(loss='hinge', optimizer=adam, metrics=[kmetr.categorical_accuracy])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) ## Minist


    # # Load the model's saved weights.
    # model.load_weights('cnn.h5')

    #train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)


    # Saveing the Model
    model.save_weights('Weight/cnn_'+str(split2)+'.h5')


    #predict first 4 images in the test set
    predit = model.predict(X_test)

    #actual results for first 4 images in test set
    print(predit[:4])


    #from sklearn.metrics import precision_recall_curve, roc_curve
    from sklearn.metrics import auc, precision_recall_curve, roc_curve
    prec, rec, thr = precision_recall_curve(y_test[:,0], predit[:,0])
    aupr_val = auc(rec, prec)
    fpr, tpr, thr = roc_curve(y_test[:,0], predit[:,0])
    auc_val = auc(fpr, tpr)
    aucListmines1.append(auc_val)
    auprListmines1.append(aupr_val)
    print(aupr_val,auc_val)

    #from sklearn.metrics import precision_recall_curve, roc_curve
    from sklearn.metrics import auc, precision_recall_curve, roc_curve
    prec, rec, thr = precision_recall_curve(y_test[:,1], predit[:,1])
    aupr_val = auc(rec, prec)
    fpr, tpr, thr = roc_curve(y_test[:,1], predit[:,1])
    auc_val = auc(fpr, tpr)
    aucListpositive1.append(auc_val)
    auprListpositive1.append(aupr_val)
    print(aupr_val , auc_val )

0 4269


Epoch 1/10
1201/1201 [==============================] - 195s 162ms/step - loss: 0.3375 - accuracy: 0.8507 - val_loss: 0.2515 - val_accuracy: 0.8937
Epoch 2/10
1201/1201 [==============================] - 177s 148ms/step - loss: 0.1960 - accuracy: 0.9194 - val_loss: 0.1983 - val_accuracy: 0.9272
Epoch 3/10
1201/1201 [==============================] - 169s 141ms/step - loss: 0.1292 - accuracy: 0.9478 - val_loss: 0.1576 - val_accuracy: 0.9440
Epoch 4/10
1201/1201 [==============================] - 168s 140ms/step - loss: 0.0946 - accuracy: 0.9615 - val_loss: 0.1474 - val_accuracy: 0.9459
Epoch 5/10
1201/1201 [==============================] - 167s 139ms/step - loss: 0.0731 - accuracy: 0.9701 - val_loss: 0.1512 - val_accuracy: 0.9471
Epoch 6/10
1201/1201 [==============================] - 166s 139ms/step - loss: 0.0603 - accuracy: 0.9747 - val_loss: 0.1799 - val_accuracy: 0.9518
Epoch 7/10
1201/1201 [==============================] - 167s 139ms/step - loss: 0.0526 - accuracy: 0.9778 - val_

Epoch 1/10
1201/1201 [==============================] - 168s 139ms/step - loss: 0.3465 - accuracy: 0.8444 - val_loss: 0.3053 - val_accuracy: 0.8618
Epoch 2/10
1201/1201 [==============================] - 167s 139ms/step - loss: 0.1980 - accuracy: 0.9167 - val_loss: 0.2096 - val_accuracy: 0.9211
Epoch 3/10
1201/1201 [==============================] - 166s 138ms/step - loss: 0.1140 - accuracy: 0.9531 - val_loss: 0.1807 - val_accuracy: 0.9356
Epoch 4/10
1201/1201 [==============================] - 168s 140ms/step - loss: 0.0731 - accuracy: 0.9711 - val_loss: 0.1779 - val_accuracy: 0.9396
Epoch 5/10
1201/1201 [==============================] - 167s 139ms/step - loss: 0.0498 - accuracy: 0.9798 - val_loss: 0.1942 - val_accuracy: 0.9482
Epoch 6/10
1201/1201 [==============================] - 166s 138ms/step - loss: 0.0354 - accuracy: 0.9862 - val_loss: 0.2273 - val_accuracy: 0.9457
Epoch 7/10
1201/1201 [==============================] - 166s 138ms/step - loss: 0.0288 - accuracy: 0.9891 - val_

Epoch 1/10
1201/1201 [==============================] - 168s 139ms/step - loss: 0.4007 - accuracy: 0.8270 - val_loss: 0.3387 - val_accuracy: 0.8501
Epoch 2/10
1201/1201 [==============================] - 167s 139ms/step - loss: 0.3216 - accuracy: 0.8660 - val_loss: 0.3244 - val_accuracy: 0.8644
Epoch 3/10
1201/1201 [==============================] - 168s 140ms/step - loss: 0.3004 - accuracy: 0.8774 - val_loss: 0.3206 - val_accuracy: 0.8611
Epoch 4/10
1201/1201 [==============================] - 168s 140ms/step - loss: 0.2808 - accuracy: 0.8850 - val_loss: 0.3188 - val_accuracy: 0.8691
Epoch 5/10
1201/1201 [==============================] - 167s 139ms/step - loss: 0.2669 - accuracy: 0.8915 - val_loss: 0.3143 - val_accuracy: 0.8726
Epoch 6/10
1201/1201 [==============================] - 169s 140ms/step - loss: 0.2594 - accuracy: 0.8979 - val_loss: 0.3111 - val_accuracy: 0.8756
Epoch 7/10
1201/1201 [==============================] - 179s 149ms/step - loss: 0.2534 - accuracy: 0.9017 - val_

Epoch 1/10
1201/1201 [==============================] - 174s 145ms/step - loss: 0.3926 - accuracy: 0.8293 - val_loss: 0.3064 - val_accuracy: 0.8602
Epoch 2/10
1201/1201 [==============================] - 192s 160ms/step - loss: 0.2949 - accuracy: 0.8772 - val_loss: 0.2878 - val_accuracy: 0.8754
Epoch 3/10
1201/1201 [==============================] - 198s 165ms/step - loss: 0.2652 - accuracy: 0.8959 - val_loss: 0.2641 - val_accuracy: 0.8963
Epoch 4/10
1201/1201 [==============================] - 192s 160ms/step - loss: 0.2401 - accuracy: 0.9091 - val_loss: 0.2571 - val_accuracy: 0.9019
Epoch 5/10
1201/1201 [==============================] - 192s 160ms/step - loss: 0.1596 - accuracy: 0.9262 - val_loss: 0.1836 - val_accuracy: 0.9124
Epoch 6/10
1201/1201 [==============================] - 318s 265ms/step - loss: 0.0987 - accuracy: 0.9403 - val_loss: 0.1619 - val_accuracy: 0.9244
Epoch 7/10
1201/1201 [==============================] - 440s 366ms/step - loss: 0.0788 - accuracy: 0.9543 - val_

...

Epoch 1/10
1201/1201 [==============================] - 173s 144ms/step - loss: 0.3617 - accuracy: 0.8386 - val_loss: 0.2802 - val_accuracy: 0.8754
Epoch 2/10
1201/1201 [==============================] - 174s 145ms/step - loss: 0.2393 - accuracy: 0.8958 - val_loss: 0.2144 - val_accuracy: 0.9138
Epoch 3/10
1201/1201 [==============================] - 178s 148ms/step - loss: 0.1705 - accuracy: 0.9291 - val_loss: 0.1940 - val_accuracy: 0.9272
Epoch 4/10
1201/1201 [==============================] - 189s 157ms/step - loss: 0.1193 - accuracy: 0.9507 - val_loss: 0.1529 - val_accuracy: 0.9424
Epoch 5/10
1201/1201 [==============================] - 197s 164ms/step - loss: 0.0799 - accuracy: 0.9659 - val_loss: 0.1531 - val_accuracy: 0.9445
Epoch 6/10
1201/1201 [==============================] - 214s 178ms/step - loss: 0.0559 - accuracy: 0.9771 - val_loss: 0.1609 - val_accuracy: 0.9543
Epoch 7/10
1201/1201 [==============================] - 208s 173ms/step - loss: 0.0416 - accuracy: 0.9825 - val_

Epoch 1/10
1201/1201 [==============================] - 251s 204ms/step - loss: 0.3835 - accuracy: 0.8318 - val_loss: 0.3257 - val_accuracy: 0.8616
Epoch 2/10
1201/1201 [==============================] - 210s 175ms/step - loss: 0.2892 - accuracy: 0.8841 - val_loss: 0.3001 - val_accuracy: 0.8810
Epoch 3/10
1201/1201 [==============================] - 227s 189ms/step - loss: 0.2558 - accuracy: 0.9016 - val_loss: 0.3072 - val_accuracy: 0.8899
Epoch 4/10
1201/1201 [==============================] - 256s 213ms/step - loss: 0.2282 - accuracy: 0.9186 - val_loss: 0.2877 - val_accuracy: 0.8944
Epoch 5/10
1201/1201 [==============================] - 252s 210ms/step - loss: 0.2155 - accuracy: 0.9297 - val_loss: 0.2781 - val_accuracy: 0.9054
Epoch 6/10
1201/1201 [==============================] - 258s 215ms/step - loss: 0.2034 - accuracy: 0.9373 - val_loss: 0.2754 - val_accuracy: 0.9101
Epoch 7/10
1201/1201 [==============================] - 275s 229ms/step - loss: 0.1974 - accuracy: 0.9433 - val_

Epoch 1/10
1201/1201 [==============================] - 216s 179ms/step - loss: 0.6932 - accuracy: 0.2146 - val_loss: 0.6931 - val_accuracy: 0.2206
Epoch 2/10
1201/1201 [==============================] - 237s 198ms/step - loss: 0.6932 - accuracy: 0.2146 - val_loss: 0.6931 - val_accuracy: 0.2206
Epoch 3/10
1201/1201 [==============================] - 207s 172ms/step - loss: 0.6932 - accuracy: 0.2146 - val_loss: 0.6931 - val_accuracy: 0.2206
Epoch 4/10
1201/1201 [==============================] - 215s 179ms/step - loss: 0.6932 - accuracy: 0.2146 - val_loss: 0.6931 - val_accuracy: 0.2206
Epoch 5/10
1201/1201 [==============================] - 232s 193ms/step - loss: 0.6932 - accuracy: 0.2146 - val_loss: 0.6931 - val_accuracy: 0.2206
Epoch 6/10
1201/1201 [==============================] - 197s 164ms/step - loss: 0.6932 - accuracy: 0.2146 - val_loss: 0.6931 - val_accuracy: 0.2206
Epoch 7/10
1201/1201 [==============================] - 194s 162ms/step - loss: 0.6932 - accuracy: 0.2146 - val_

Epoch 1/10
1201/1201 [==============================] - 191s 158ms/step - loss: 0.3884 - accuracy: 0.8315 - val_loss: 0.3086 - val_accuracy: 0.8651
Epoch 2/10
1201/1201 [==============================] - 194s 162ms/step - loss: 0.2887 - accuracy: 0.8816 - val_loss: 0.3018 - val_accuracy: 0.8679
Epoch 3/10
1201/1201 [==============================] - 185s 154ms/step - loss: 0.1852 - accuracy: 0.9073 - val_loss: 0.1786 - val_accuracy: 0.9037
Epoch 4/10
1201/1201 [==============================] - 177s 147ms/step - loss: 0.1199 - accuracy: 0.9322 - val_loss: 0.1758 - val_accuracy: 0.9166
Epoch 5/10
1201/1201 [==============================] - 174s 145ms/step - loss: 0.0885 - accuracy: 0.9455 - val_loss: 0.1765 - val_accuracy: 0.9180
Epoch 6/10
1201/1201 [==============================] - 173s 144ms/step - loss: 0.0732 - accuracy: 0.9530 - val_loss: 0.1708 - val_accuracy: 0.9253
Epoch 7/10
1201/1201 [==============================] - 174s 144ms/step - loss: 0.0591 - accuracy: 0.9664 - val_

Epoch 1/10
1201/1201 [==============================] - 176s 146ms/step - loss: 0.3389 - accuracy: 0.8506 - val_loss: 0.2538 - val_accuracy: 0.8928
Epoch 2/10
1201/1201 [==============================] - 174s 145ms/step - loss: 0.1774 - accuracy: 0.9277 - val_loss: 0.1761 - val_accuracy: 0.9267
Epoch 3/10
1201/1201 [==============================] - 175s 146ms/step - loss: 0.1086 - accuracy: 0.9577 - val_loss: 0.1607 - val_accuracy: 0.9452
Epoch 4/10
1201/1201 [==============================] - 175s 145ms/step - loss: 0.0753 - accuracy: 0.9711 - val_loss: 0.1522 - val_accuracy: 0.9462
Epoch 5/10
1201/1201 [==============================] - 175s 145ms/step - loss: 0.0555 - accuracy: 0.9788 - val_loss: 0.1412 - val_accuracy: 0.9511
Epoch 6/10
1201/1201 [==============================] - 173s 144ms/step - loss: 0.0432 - accuracy: 0.9826 - val_loss: 0.1449 - val_accuracy: 0.9527
Epoch 7/10
1201/1201 [==============================] - 174s 145ms/step - loss: 0.0377 - accuracy: 0.9849 - val_

In [14]:
from scipy.stats import sem, t
from scipy import mean
confidence = 0.95
# data = [1, 2, 3, 4, 5]

naucList = len(aucListpositive1)
maucList = mean(aucListpositive1)
std_erraucList = sem(aucListpositive1)
haucList = std_erraucList * t.ppf((1 + confidence) / 2, naucList - 1)


nauprList = len(auprListpositive1)
mauprList = mean(auprListpositive1)
std_errauprList = sem(auprListpositive1)
hauprList = std_errauprList * t.ppf((1 + confidence) / 2, nauprList  - 1)



print("positive1: "+ str(maucList) + '  -+' + str(haucList), str(mauprList) + '  -+' + str(hauprList))

positive1: 0.8978377250844017  -+0.10601405895824866 0.9681268243661777  -+0.035940383764222504


C:\Users\Akhod\AppData\Local\Temp\ipykernel_1504\666640578.py:7: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  maucList = mean(aucListpositive1)
C:\Users\Akhod\AppData\Local\Temp\ipykernel_1504\666640578.py:13: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  mauprList = mean(auprListpositive1)


In [15]:
mypd = []
mypd = pd.DataFrame(mypd)
mypd['AUC'],mypd['AUPR'] = aucListpositive1, auprListpositive1
mypd.to_csv('selectedModelresult(+1).csv')

In [16]:
from scipy.stats import sem, t
from scipy import mean
confidence = 0.95
# data = [1, 2, 3, 4, 5]

naucList = len(aucListmines1)
maucList = mean(aucListmines1)
std_erraucList = sem(aucListmines1)
haucList = std_erraucList * t.ppf((1 + confidence) / 2, naucList - 1)


nauprList = len(auprListmines1)
mauprList = mean(auprListmines1)
std_errauprList = sem(auprListmines1)
hauprList = std_errauprList * t.ppf((1 + confidence) / 2, nauprList  - 1)



print("mines1: "+ str(maucList) + '  -+' + str(haucList), str(mauprList) + '  -+' + str(hauprList))

mines1: 0.897001638449514  -+0.10731220070047721 0.8700366809493786  -+0.07776600964645441


C:\Users\Akhod\AppData\Local\Temp\ipykernel_1504\1263506168.py:7: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  maucList = mean(aucListmines1)
C:\Users\Akhod\AppData\Local\Temp\ipykernel_1504\1263506168.py:13: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  mauprList = mean(auprListmines1)


In [17]:
mypd = []
mypd = pd.DataFrame(mypd)
mypd['AUC'],mypd['AUPR'] = aucListmines1, auprListmines1
mypd.to_csv('selectedModelresult(-1).csv')